### Part 1 - a
Loading in the dataset and defining classification functions

In [137]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

iris = load_iris()

def TrainDataWithKFolds(estimators, folds):
    kfold = KFold(n_splits=10, random_state=11, shuffle=True)

    duck = {}
    
    #Now execute each model and return the 
    for name, obj in estimators.items():
        kfold = KFold(n_splits=folds, random_state=11, shuffle=True)
        scores = cross_val_score(estimator=obj, 
            X=iris.data, y=iris.target, cv=kfold)

        duck[name] = scores.mean()
        print(f'With k = {folds}, '
            f'{name:>20}: ' + 
            f'mean accuracy={scores.mean():.2%}; ' +
            f'standard deviation={scores.std():.2%}')

    return duck

### Part 1 - b
K- Closest Neighbor



Neural Network using multi-layer perceptron

In [143]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes  import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

estimators = {
    'KNeighborsClassifier': KNeighborsClassifier(), 
    'SGD': SGDClassifier(loss="hinge", penalty="l2", max_iter=1000),
    'GaussianNB': GaussianNB()}

twoFolds = TrainDataWithKFolds(estimators, 2)
twnentyFolds = TrainDataWithKFolds(estimators, 20)

maxClassifier = max(max(twoFolds, key=twoFolds.get), max(twnentyFolds, key=twnentyFolds.get))
    #estimators[maxClassifier], max(max(twoFolds.values()), max(twnentyFolds.values()))
print(f"The best estimator for this dataset is {estimators[maxClassifier]}")

With k = 2, KNeighborsClassifier: mean accuracy=95.33%; standard deviation=0.67%
With k = 2,                  SGD: mean accuracy=87.33%; standard deviation=2.00%
With k = 2,           GaussianNB: mean accuracy=95.33%; standard deviation=3.33%
With k = 20, KNeighborsClassifier: mean accuracy=96.70%; standard deviation=5.74%
With k = 20,                  SGD: mean accuracy=81.70%; standard deviation=15.39%
With k = 20,           GaussianNB: mean accuracy=94.91%; standard deviation=7.39%
The best estimator for this dataset is KNeighborsClassifier()


In [ ]:
#We need pandas for the DateFrame, seaborn for the heatmap, and plt for the figure and color map 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix




confusion = confusion_matrix(y_true=expected_labels, y_pred=predicted_labels)
# confusion_df = pd.DataFrame(confusion, index=range(10), columns=range(10))